### Basic import

In [2]:
import rosbag
import roslib
import cv2
import numpy as np
import pandas as pd
from cv_bridge import CvBridge, CvBridgeError
from rospy import Subscriber
from sensor_msgs.msg import Image
from opt_msgs.msg import DetectionArray
import message_filters
from message_filters import TimeSynchronizer, Subscriber
import socket, json

In [3]:
# Create bag to process
bag = rosbag.Bag('../../bags/solo.bag')
print bag

path:        ../../bags/solo.bag
version:     2.0
duration:    29.9s
start:       May 16 2017 13:25:15.41 (1494933915.41)
end:         May 16 2017 13:25:45.27 (1494933945.27)
size:        4.2 GB
messages:    2056
compression: none [754/754 chunks]
types:       opt_msgs/DetectionArray [2be460a0a39ff0f809695e59da9d469e]
             opt_msgs/TrackArray     [4106b2da5c66b9f239aada7912e67b8f]
             sensor_msgs/Image       [060021388200f6f0f447d0fcd9c64743]
             sensor_msgs/PointCloud2 [1158d486dd51d683ce2f1be655c3c181]
topics:      /Kinect_Laurene/depth_lowres/image    262 msgs @ 11.7 Hz : sensor_msgs/Image      
             /Kinect_Laurene/depth_lowres/points   230 msgs @  8.7 Hz : sensor_msgs/PointCloud2
             /Kinect_Laurene/rgb_lowres/image      262 msgs @ 10.3 Hz : sensor_msgs/Image      
             /detector/detections                  651 msgs @ 19.7 Hz : opt_msgs/DetectionArray (2 connections)
             /tracker/tracks                       651 msgs @ 19

In [4]:
# create bridge between CV and ROS
br = CvBridge()

In [5]:
bag.read_messages(topics=['/Kinect_Laurene/depth_lowres/points'])

<generator object read_messages at 0x7f84b8c3eb90>

In [39]:
# Create array for cloudPoint
cloudTime = []
cloudSeq = []
for topic, msg, t in bag.read_messages(topics=['/Kinect_Laurene/depth_lowres/points']):
    time = msg.header.stamp
    cloudTime.append(int(str(time)))
    seq = msg.header.seq
    cloudSeq.append(seq)

In [40]:
cloudTime

[1494933915296689349,
 1494933915362898146,
 1494933915445632482,
 1494933915631015687,
 1494933915736506531,
 1494933915909519823,
 1494933916016937893,
 1494933916042489798,
 1494933916233920932,
 1494933916266464260,
 1494933916420375468,
 1494933916595584817,
 1494933916680907316,
 1494933916766370616,
 1494933917006193941,
 1494933917075297851,
 1494933917296817504,
 1494933917465115374,
 1494933917503182061,
 1494933917643314061,
 1494933917884096560,
 1494933917955353581,
 1494933918064590441,
 1494933918243399625,
 1494933918301063135,
 1494933918344869592,
 1494933918421701052,
 1494933918448261110,
 1494933918528274934,
 1494933918685421007,
 1494933918940631024,
 1494933919058490313,
 1494933919172382076,
 1494933919410175761,
 1494933919738318646,
 1494933919808658444,
 1494933919892936685,
 1494933920005137600,
 1494933920077187681,
 1494933920177126261,
 1494933920425175443,
 1494933920543197459,
 1494933920624806610,
 1494933920755088662,
 1494933920988978176,
 149493392

In [41]:
# Create a txt file with detection msg separate by 2 new lines
file = open("/home/sabrine/notebook/messages/detect_solo_L.txt","w")
detectTime = []
detectSeq = []

for topic, msg, t in bag.read_messages(topics=['/detector/detections']):
    for e in str(msg).split('\n'):
        if 'Laurene' in e:
            time = msg.header.stamp
            detectTime.append(int(str(time)))
            seq = msg.header.seq
            detectSeq.append(seq)
            file.write(str(msg) + "\n \n")

file.close()

In [42]:
len(detectTime)

203

In [43]:
cloud = pd.DataFrame(data=cloudTime, index=cloudSeq)

In [56]:
cloud['seq'] = cloud.index

In [30]:
detect = pd.DataFrame(data=detectTime, index=detectSeq)

In [44]:
def find_nearest(array,value):
    idx = (np.abs(array-value)).argmin()
    return array[idx]

In [51]:
match_cloud= []
for index, row in detect.iterrows():
    #print index
    #print row[0]
    cloudSeqMatched = find_nearest(np.asarray(cloudTime), row[0])
    match_cloud.append(cloudSeqMatched)

In [54]:
detect['match_cloud'] = match_cloud

In [59]:
detect['seq'] = detect.index

In [61]:
synchronize = pd.merge(detect, cloud, how='inner', left_on='match_cloud', right_on=0,
         left_index=False, right_index=False, sort=True,
         suffixes=('_msg', '_cloud'), copy=True, indicator=False)

In [64]:
synchronize

,0_msg,match_cloud,seq_msg,0_cloud,seq_cloud
0,1494933915010274000,1494933915296689349,5894,1494933915296689349,7374
1,1494933915256237000,1494933915296689349,5895,1494933915296689349,7374
2,1494933915296689000,1494933915296689349,5896,1494933915296689349,7374
3,1494933915362898000,1494933915362898146,5897,1494933915362898146,7375
4,1494933915445632000,1494933915445632482,5898,1494933915445632482,7376
5,1494933915631015000,1494933915631015687,5899,1494933915631015687,7377
6,1494933915736506000,1494933915736506531,5900,1494933915736506531,7378
7,1494933915909519000,1494933915909519823,5901,1494933915909519823,7379
8,1494933916016937000,1494933916016937893,5902,1494933916016937893,7380
9,1494933916233920000,1494933916233920932,5903,1494933916233920932,7382


In [67]:
synchronize['seq_cloud']

0      7374
1      7374
2      7374
3      7375
4      7376
5      7377
6      7378
7      7379
8      7380
9      7382
10     7383
11     7384
12     7385
13     7386
14     7388
15     7389
16     7390
17     7391
18     7392
19     7393
20     7394
21     7395
22     7396
23     7397
24     7398
25     7399
26     7400
27     7401
28     7402
29     7403
       ... 
173    7568
174    7571
175    7572
176    7573
177    7574
178    7576
179    7577
180    7579
181    7580
182    7581
183    7582
184    7583
185    7584
186    7585
187    7586
188    7587
189    7588
190    7589
191    7590
192    7592
193    7593
194    7594
195    7595
196    7596
197    7597
198    7598
199    7599
200    7600
201    7601
202    7603
Name: seq_cloud, dtype: int64

In [68]:
synchronize.index = synchronize['seq_msg']

In [70]:
synchronize['seq_cloud']

seq_msg
5894    7374
5895    7374
5896    7374
5897    7375
5898    7376
5899    7377
5900    7378
5901    7379
5902    7380
5903    7382
5904    7383
5905    7384
5906    7385
5907    7386
5908    7388
5909    7389
5910    7390
5911    7391
5912    7392
5913    7393
5914    7394
5915    7395
5916    7396
5917    7397
5918    7398
5919    7399
5920    7400
5921    7401
5922    7402
5923    7403
        ... 
6067    7568
6068    7571
6069    7572
6070    7573
6071    7574
6072    7576
6073    7577
6074    7579
6075    7580
6076    7581
6077    7582
6078    7583
6079    7584
6080    7585
6081    7586
6082    7587
6083    7588
6084    7589
6085    7590
6086    7592
6087    7593
6088    7594
6089    7595
6090    7596
6091    7597
6092    7598
6093    7599
6094    7600
6095    7601
6096    7603
Name: seq_cloud, dtype: int64